In [ ]:
import imageio
import sh
import ffmpeg
import subprocess

In [2]:
import numpy as np

from glitch import (
    move_blocks,
    salt_and_pepper,
    move_channels_random,
    get_video_size,
    start_ffmpeg_writer,
    start_ffmpeg_reader,
    read_frame
)

In [3]:
in_fname = 'IMG_7443.mov'
out_fname = 'IMG_7443_glitch.mp4'
width, height = get_video_size(in_fname)
reader = start_ffmpeg_reader(in_fname)
writer = start_ffmpeg_writer(out_fname, width, height)
while True:
    frame = read_frame(reader, width, height)
    if frame is None:
        # no more frames
        break
    
    # process frame
    roll = np.random.randint(0,6)
    # 0 -> move channels
    # 1 -> swap blocks
    # 2 -> both
    # 3+ -> nothing
    
    roll_noise = np.random.randint(0,4)
    # if 0 or 1 noise
    
    frame = frame.copy()
    if roll in [0, 2]:
        frame = move_channels_random(frame, -15, 15)
    if roll in [1, 2]:
        blocksize_x = np.random.randint(min(width, height) * .1, max(width, height) * .5)
        blocksize_y = np.random.randint(min(width, height) * .1, max(width, height) * .5)
        frame = move_blocks(frame, blocksize=(blocksize_x, blocksize_y), num_blocks=3, per_channel=True)
    
    if roll_noise in [0, 1]:
        frame = salt_and_pepper(frame,0.5, .02)

    writer.stdin.write(frame.astype(np.uint8).tobytes())

# cleanup
reader.wait()
writer.stdin.close()
writer.wait()
print('done')

done
